In [1]:
def get_path(dataset_name,env_name='colab'):
    """
    This function is used to return the path of the dataset you want to use. 
    
    @params:
    dataset_name: the name of the dataset. 
    env_name: it has two values either local, or colab the default is colab
    """
    prefix = 'https://raw.githubusercontent.com/mohamed-ashry7/Data-Engineering-Lab/main/Datasets/'
    if env_name == 'colab':
        return prefix+dataset_name
    else:
        return f'../Datasets/{dataset_name}'

In [2]:
!sudo apt update


Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Get:15 http://ppa

In [3]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


In [4]:
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz



In [5]:
# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz


In [6]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [7]:
!pip install -q findspark

In [8]:
import findspark
findspark.init() 

In [9]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 198 kB 47.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=819775c61f9f9c5135ec000e03291866931be44dfef316a8beba2e82fd54a0f8
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


To work on data using Apache Spark, we first need to instantiate a spark session, this session is going to interact with the Spark Context to manage the data processing.

In [10]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
spark = SparkSession.builder.appName("Challenge 1").getOrCreate()  # use all cores of your CPU - each core has thread

#### **Challenge 1 - SQL and PySpark Querying**

Write a PySpark code to read all the files in the `by-day` folder, and displays a table that shows the ID of the customer and the the total amount they paid for all their purchases. Make sure to drop any stock that has the word *WATER* in it before calculating the total price.

In [11]:
#Your answer here
url = get_path("by-day.zip")
!wget $url


--2022-01-07 20:15:53--  https://raw.githubusercontent.com/mohamed-ashry7/Data-Engineering-Lab/main/Datasets/by-day.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8194140 (7.8M) [application/zip]
Saving to: ‘by-day.zip’

by-day.zip          100%[===================>]   7.81M  --.-KB/s    in 0.08s   

2022-01-07 20:15:54 (93.3 MB/s) - ‘by-day.zip’ saved [8194140/8194140]



In [12]:
!unzip by-day.zip

Archive:  by-day.zip
  inflating: by-day/2010-12-01.csv   
  inflating: by-day/2010-12-02.csv   
  inflating: by-day/2010-12-03.csv   
  inflating: by-day/2010-12-05.csv   
  inflating: by-day/2010-12-06.csv   
  inflating: by-day/2010-12-07.csv   
  inflating: by-day/2010-12-08.csv   
  inflating: by-day/2010-12-09.csv   
  inflating: by-day/2010-12-10.csv   
  inflating: by-day/2010-12-12.csv   
  inflating: by-day/2010-12-13.csv   
  inflating: by-day/2010-12-14.csv   
  inflating: by-day/2010-12-15.csv   
  inflating: by-day/2010-12-16.csv   
  inflating: by-day/2010-12-17.csv   
  inflating: by-day/2010-12-19.csv   
  inflating: by-day/2010-12-20.csv   
  inflating: by-day/2010-12-21.csv   
  inflating: by-day/2010-12-22.csv   
  inflating: by-day/2010-12-23.csv   
  inflating: by-day/2011-01-04.csv   
  inflating: by-day/2011-01-05.csv   
  inflating: by-day/2011-01-06.csv   
  inflating: by-day/2011-01-07.csv   
  inflating: by-day/2011-01-09.csv   
  inflating: by-day/2011-01-1

In [13]:
spark_df =spark.read.format('csv').option("header", "true").option("inferSchema","true").load("by-day/*.csv")

Solve the same question using raw SQL.

In [14]:
#Your answer here
spark_df.createOrReplaceTempView("ByDay")

### Using SQL

In [15]:
query = spark.sql("""
  SELECT CustomerID,  sum(UnitPrice * Quantity) AS Total
  FROM ByDay
  WHERE Description NOT LIKE '%WATER%'
  GROUP BY CustomerID
""")

In [16]:
query.show()

+----------+------------------+
|CustomerID|             Total|
+----------+------------------+
|   14452.0|264.44000000000005|
|   16916.0| 565.3599999999999|
|   17633.0|1242.3400000000001|
|   14768.0|             139.5|
|   13094.0|1708.8600000000001|
|   17884.0|            674.95|
|   16596.0|             220.3|
|   13607.0| 648.3099999999998|
|   14285.0|1589.0100000000002|
|   16561.0|            481.87|
|   13956.0|1026.4199999999998|
|   13533.0|            270.79|
|   16629.0| 417.7299999999999|
|   17267.0|312.96999999999997|
|   13918.0|           1212.84|
|   18114.0|            216.35|
|   14473.0|234.34000000000003|
|   14024.0|             327.7|
|   12493.0|            393.39|
|   15776.0|241.62000000000003|
+----------+------------------+
only showing top 20 rows



### Using Pyspark

In [17]:
from pyspark.sql.functions import col
df1 = spark_df.select(spark_df.columns).where(~col("Description").like("% WATER %"))
df1.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   580538|    23084|  RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|
|   580538|    23077| DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22906|12 MESSAGE CARDS ...|      24|2011-12-05 08:38:00|     1.65|   14075.0|United Kingdom|
|   580538|    21914|BLUE HARMONICA IN...|      24|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22467|   GUMBALL COAT RACK|       6|2011-12-05 08:38:00|     2.55|   14075.0|United Kingdom|
|   580538|    23126|FELTCRAFT GIRL AM...|       8|2011-12-05 08:38:00|     4.95|   14075.0|United Kingdom|
|   580538|    21833|CAMOUFL

In [25]:
from pyspark.sql.functions import desc
df2 = df1.selectExpr("CustomerID", "(UnitPrice * Quantity) as total_cost").groupby(col("CustomerID")).sum("total_cost")

In [26]:
df2.show()

+----------+------------------+
|CustomerID|   sum(total_cost)|
+----------+------------------+
|   14452.0|264.44000000000005|
|   16916.0| 565.3599999999999|
|   17633.0|1242.3400000000001|
|   14768.0|             139.5|
|   13094.0|1708.8600000000001|
|   17884.0|            674.95|
|   16596.0|             220.3|
|   13607.0| 648.3099999999998|
|   14285.0|1589.0100000000002|
|   16561.0|            481.87|
|   13956.0|1026.4199999999998|
|   13533.0|            270.79|
|   16629.0| 417.7299999999999|
|   17267.0|312.96999999999997|
|   13918.0|           1212.84|
|   18114.0|            216.35|
|   14473.0|234.34000000000003|
|   14024.0|             327.7|
|   12493.0|416.78999999999996|
|   15776.0|241.62000000000003|
+----------+------------------+
only showing top 20 rows



### **Challenge 2 - Pipeline**

We use Pipeline to specify our machine learning workflow. A Pipeline’s stages are specified as an ordered array.

Applying the following tasks to create your ML pipeline :

1. First, read all the files in the `by-day` folder.
2. Next, create a pipeline using the `Pipeline()` function, in which the indexer is applied followed by the assembler.
3. Afterwards, fit and transform the data at hand using the pipeline we created which includes: StringIndexer and VectorAssembler. 


In [19]:
# Your answer here
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
sindexer = StringIndexer().setInputCol("day_of_week").setOutputCol("day_of_week_indexed")
va = VectorAssembler().setInputCols(["UnitPrice","Quantity", "day_of_week_indexed"]).setOutputCol("Unit-Quantity-day_of_week")

In [20]:
from pyspark.sql.functions import date_format,col
my_df = df1.withColumn("day_of_week",date_format(col("InvoiceDate"),"EEEE"))

In [21]:
my_df.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|day_of_week|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+
|   580538|    23084|  RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|     Monday|
|   580538|    23077| DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|     Monday|
|   580538|    22906|12 MESSAGE CARDS ...|      24|2011-12-05 08:38:00|     1.65|   14075.0|United Kingdom|     Monday|
|   580538|    21914|BLUE HARMONICA IN...|      24|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|     Monday|
|   580538|    22467|   GUMBALL COAT RACK|       6|2011-12-05 08:38:00|     2.55|   14075.0|United Kingdom|     Monday|
|   580538|    23126|FELTCRAFT GIRL AM..

In [22]:
from pyspark.ml import Pipeline
my_pl = Pipeline().setStages([sindexer,va])

In [23]:
fittedpl = my_pl.fit(my_df)
transpl = fittedpl.transform(my_df)

In [24]:
transpl.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+-------------------+-------------------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|day_of_week|day_of_week_indexed|Unit-Quantity-day_of_week|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+-------------------+-------------------------+
|   580538|    23084|  RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|     Monday|                2.0|          [1.79,48.0,2.0]|
|   580538|    23077| DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|     Monday|                2.0|          [1.25,20.0,2.0]|
|   580538|    22906|12 MESSAGE CARDS ...|      24|2011-12-05 08:38:00|     1.65|   14075.0|United Kingdom|     Monday|                2.0|          [1.65,24.0,2.0]|
|   